In [ ]:
import os
import pandas as pd
import numpy as np
import pygeos
import geopandas as gpd
from tqdm import tqdm
import pyproj

In [ ]:
def reproject_assets(df_ds,current_crs="epsg:4326",approximate_crs = "epsg:28992"):
    """[summary]

    Args:
        df_ds ([type]): [description]
        current_crs (str, optional): [description]. Defaults to "epsg:3857".
        approximate_crs (str, optional): [description]. Defaults to "epsg:4326".

    Returns:
        [type]: [description]
    """    

    geometries = df_ds['geometry']
    coords = pygeos.get_coordinates(geometries)
    transformer=pyproj.Transformer.from_crs(current_crs, approximate_crs,always_xy=True)
    new_coords = transformer.transform(coords[:, 0], coords[:, 1])
    
    return pygeos.set_coordinates(geometries.copy(), np.array(new_coords).T) 

In [ ]:
# SET PATHS
province = 'zeeland'
building_path = os.path.join("C:\\Data",'OSM','feather')
cbs_path = os.path.join("C://","Data","Bodemgebruik")

building_province_path = os.path.join(building_path,'{}.ft'.format(province))

In [ ]:
def get_use(x,omschrijving,tree):
    try:
        return omschrijving[tree.query(x,predicate='intersects')][0]
    except:
        return 'overig landgebruik'
    
def update_building_information(province):
    building_path = os.path.join("C:\\Data",'OSM','feather')
    cbs_path = os.path.join("C://","Data","Bodemgebruik")

    building_province_path = os.path.join(building_path,'{}.ft'.format(province))

    # read df_buildings
    df_buildings = pd.read_feather(building_province_path)
    df_buildings.geometry = pygeos.from_wkb(df_buildings.geometry.values)
    df_buildings.geometry = reproject_assets(df_buildings)

    BBG2015 = gpd.read_file(os.path.join(cbs_path,'BBG2015.shp'))

    print('BBG loaded for {}'.format(province))

    landuse_to_keep = ['Spoorweg', 'Hoofdweg', 'Vliegveld', 'Woongebied',
           'Detailhandel en horeca', 'Openbare voorziening',
           'Sociaal-culturele voorziening', 'Bedrijfsterrein', 'Stortplaats',
           'Wrakkenopslagplaats', 'Begraafplaats', 'Delfstofwinplaats',
           'Bouwterrein', 'Semi verhard overig terrein', 'Park en plantsoen',
           'Sportterrein', 'Volkstuin', 'Dagrecreatief terrein',
           'Verblijfsrecreatie', 'Glastuinbouw', 'Overig agrarisch gebruik']

    BBG2015 = BBG2015.loc[BBG2015.Omschrijvi.isin(landuse_to_keep)].reset_index()    

    df_BBG = pd.DataFrame(BBG2015[['Omschrijvi','geometry']].copy())
    df_BBG.geometry = pygeos.from_shapely(df_BBG.geometry)

    province_bbox = pygeos.envelope(pygeos.geometrycollections(df_buildings.geometry.values))
    province_BBG = df_BBG.loc[pygeos.intersects(df_BBG.geometry,province_bbox)].reset_index()
    
    omschrijving = province_BBG.Omschrijvi.values

    tree = pygeos.STRtree(province_BBG.geometry)

    df_buildings['centroid'] = pygeos.centroid(df_buildings.geometry)
        
    tqdm.pandas(desc=province)
    df_buildings['BBG'] = df_buildings['centroid'].progress_apply(lambda x: get_use(x,omschrijving,tree))

    df_buildings.geometry = reproject_assets(df_buildings,current_crs="epsg:28992",approximate_crs = "epsg:4326")
    df_buildings = df_buildings.drop(['centroid'],axis=1)
    df_buildings.geometry = pygeos.to_wkb(df_buildings.geometry)

    df_buildings.to_feather(building_province_path)

    return df_buildings

In [ ]:
test = update_building_information(province)

In [ ]:
test.BBG.unique()